In [6]:
import pandas as pd

# Load the uploaded CSV file to inspect its structure
option_file_path = '../data/test_data/test-config-repo_options.csv'
option_data = pd.read_csv(option_file_path)


################################################################################################
# Create the basis graph data for a given project
################################################################################################


# Create a list of unique nodes
concepts = option_data['Concept'].unique()
artifacts = option_data['File Path'].unique()
options = option_data[['File Path', 'Option', 'Values', 'Changed internally', 'Changed globally']].apply(tuple, axis=1)

# Define nodes
nodes = [{'id': concept, 'type': 'concept'} for concept in concepts]
nodes += [{'id': artifact, 'type': 'artifact'} for artifact in artifacts]
nodes += [{'id': f"{option[0]}:{option[1]}", 'values': str(option[2]), 'changed_internally': option[3], 'changed_globally': option[4], 'type': 'option', } for option in options]

# Define links
links = []

# Link concepts to artifacts
for _, row in option_data.iterrows():
    links.append({'source': row['Concept'], 'target': row['File Path'], 'type': 'concept-artifact'})
    
# Link artifacts to options
for _, row in option_data.iterrows():
    artifact = row["File Path"]
    option = row["Option"]
    links.append({'source': row['File Path'], 'target': f"{artifact}:{option}", 'type': 'artifact-option'})


################################################################################################
# Add additional links to the graph data (Co-Changes of concepts)
################################################################################################
# Links concepts to concepts
concept_relations_path = '../data/concept_cochanges/test-config-repo_concept_cochanges.csv'  # Update with your file path
concept_relations = pd.read_csv(concept_relations_path)

for _, row in concept_relations.iterrows():
    source_concept = row['Concept1']
    target_concept = row['Concept2']
    weight = row['Percentage Internally']
    count = row['Changed Internally']
    
    # Create a link for the concept relationship
    links.append({
        'source': source_concept,
        'target': target_concept,
        'type': 'concept-concept',
        'weight': weight, # Add weight to use for link strength or color
        'count': count  # Add count to use for
    })

################################################################################################
# Add additional links to the graph data (Co-Changes of config file)
################################################################################################
# TODO: Add code to link artifacts based on their co-changes


################################################################################################
# Add additional links to the graph data (Co-Changes of config options)
################################################################################################
# TODO: Add code to link options based on their co-changes



# Output the data
graph_data = {'nodes': nodes, 'links': links}

# Save the graph data as JSON for D3.js
import json
output_path = '../data/test_data/graph_data/test-config-repo_graph_data.json'
with open(output_path, 'w') as f:
    json.dump(graph_data, f)

output_path

'../data/test_data/graph_data/test-config-repo_graph_data.json'